In [ ]:
%pip install isodate

In [ ]:

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import isodate  # For converting duration format like PT11M7S

# Load the dataset
df = pd.read_csv('/content/primary_data.csv')  # Change the file name as needed

# Clean column names (remove extra spaces)
df.columns = df.columns.str.strip()

# Convert Duration column from string to minutes
def convert_duration(duration_str):
    try:
        duration = isodate.parse_duration(duration_str)
        return duration.total_seconds() / 60  # Convert to minutes
    except:
        return np.nan

df['Duration'] = df['Duration'].apply(convert_duration)

# Drop rows with unconvertible duration values
df.dropna(subset=['Duration'], inplace=True)

# Create a "Popularity" column based on number of views
threshold = df['Views'].quantile(0.75)  # Use top 25% as popular
df['Popularity'] = df['Views'].apply(lambda x: 1 if x >= threshold else 0)

# Select features that may influence popularity
features = ['Duration', 'Likes', 'Comments', 'Language', 'Topic', 'Publication Time', 'Region']
X = df[features]
y = df['Popularity']

# Convert categorical values to numeric
label_cols = ['Language', 'Topic', 'Publication Time', 'Region']
le = LabelEncoder()
for col in label_cols:
    X[col] = le.fit_transform(X[col])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Prediction and evaluation
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Display first few rows of the dataset
print(df.head())


<ipython-input-4-52cd818eb302>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-4-52cd818eb302>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-4-52cd818eb302>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       377
           1       0.90      0.83      0.86       126

    accuracy                           0.93       503
   macro avg       0.92      0.90      0.91       503
weighted avg       0.93      0.93      0.93       503

